Get all datasets and the according attribute values which contain (at least) one link without _https_ which also works with the replaced protocol.

In [ ]:
! pip install urlextract

In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast

In [3]:
response = requests.get('https://portal.collab.admin.ch/sites/805-EA/RP%20GI/BFE-D-0103_old.xlsx')
response

<Response [403]>

In [4]:
a = [['Allgemeinstrom', 'S110kV', 'Freileitung', 232323, '12.12.2020'],
     ['Allgemeinstrom', 'S110kV', 'Kabelleitung', 100, '12.12.2020'],
     ['Allgemeinstrom', 'S20kV', 'Freileitung', 5, '12.12.2020'],
     ['Allgemeinstrom', 'S20kV', 'Kabelleitung', 7, '12.12.2020']]

dfLeistungKPI = pd.DataFrame(a, columns=['Stromtyp', 'Spannung', 'Leitung', 'Laenge', 'Datum'])
dfLeistungKPI

,Stromtyp,Spannung,Leitung,Laenge,Datum
0,Allgemeinstrom,S110kV,Freileitung,232323,12.12.2020
1,Allgemeinstrom,S110kV,Kabelleitung,100,12.12.2020
2,Allgemeinstrom,S20kV,Freileitung,5,12.12.2020
3,Allgemeinstrom,S20kV,Kabelleitung,7,12.12.2020


In [24]:
colNames = []
values   = []

for current in dfLeistungKPI['Spannung'].unique():
    
    # get all information about a current in a small frame
    tmp = dfLeistungKPI[dfLeistungKPI.Spannung == current]
    
    # get the value for 'Kabelleitung'
    colNames.append(current + '_Kabel')
    kabelleitung = tmp[tmp.Leitung == 'Kabelleitung']['Laenge'].values[0]
    values.append(kabelleitung)
    
    # get the sum of both types
    colNames.append(current + '_Total')
    sumOfBoth = tmp['Laenge'].sum()
    values.append(sumOfBoth)

    
dfResults = pd.DataFrame(values).T
dfResults.columns = colNames
df

In [25]:
values

[100, 232423, 7, 12]

In [29]:
df = pd.DataFrame(values).T
df.columns = colNames
df

,S110kV_Kabel,S110kV_Total,S20kV_Kabel,S20kV_Total
0,100,232423,7,12


In [13]:
values

[         Stromtyp Spannung       Leitung  Laenge       Datum
 1  Allgemeinstrom   S110kV  Kabelleitung     100  12.12.2020,
 232423,
          Stromtyp Spannung       Leitung  Laenge       Datum
 3  Allgemeinstrom    S20kV  Kabelleitung       7  12.12.2020,
 12]

In [11]:
d = len(cols) * ['abc']
d

['abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc',
 'abc']

In [17]:
df = pd.DataFrame(d).T
df.columns = cols
df

,ID,Titel,Beschreibung,Organisationseinheit,Kontakt,Publikationsdatum,Schlüsselworte,Aktualisierungsintervall,Letze_Aktualisierung,Dateninhaber,Status,Klassifizierung,Offenheit,Personendaten,Stammdaten,Referenzdaten,Datentyp
0,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc,abc


In [18]:
df.to_json('outputs/json_test.json', orient='records')

In [12]:
df = pd.DataFrame(d, columns=['ID', 'Titel', 'Beschreibung', 'Organisationseinheit', 'Kontakt',
                           'Publikationsdatum', 'Schlüsselworte', 'Aktualisierungsintervall',
                           'Letze_Aktualisierung', 'Dateninhaber', 'Status', 'Klassifizierung',
                           'Offenheit', 'Personendaten', 'Stammdaten', 'Referenzdaten', 'Datentyp'])
df

ValueError: Shape of passed values is (17, 1), indices imply (17, 17)

In [8]:
cols=['ID', 'Titel', 'Beschreibung', 'Organisationseinheit', 'Kontakt',
                           'Publikationsdatum', 'Schlüsselworte', 'Aktualisierungsintervall',
                           'Letze_Aktualisierung', 'Dateninhaber', 'Status', 'Klassifizierung',
                           'Offenheit', 'Personendaten', 'Stammdaten', 'Referenzdaten', 'Datentyp']
len(cols)

17

In [68]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

In [69]:
def check_url(url):
  try:
    get = requests.head(url)

    if get.status_code < 400:
      return True
    else:
      return False

  except requests.exceptions.RequestException as e:
    raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [70]:
def addHTTPS(url):
  adjusted = url.replace("http:", "https:" )
  adjusted = adjusted.replace("www", "https://www" )

  return adjusted.replace("https://https://www", "https://www" )

In [ ]:
def addHTTP(url):
  adjusted = url.replace("www", "http://www")
  
  return adjusted.replace("http://http://www", "http://www")

In [ ]:
def getURLs(urls):
  
  links = []

  for url in urls:

    url_modified    = ''
    # only store links which are faulty or need a protocol update (https)
    needsToBeListed = False

    if 'https' in url:
      working = check_url(url)

      if not working:
        needsToBeListed = True


    else:
      
      url_modified = addHTTPS(url)
      working = check_url(url_modified)

      if working:
        needsToBeListed = True
        
      else:
        # does not change http: urls
        url_modified = addHTTPS(url)
        working = check_url(url)

        if not working:
          url_modified = ''
          needsToBeListed = True

    if not needsToBeListed:
      continue
    

    links.append([url, url_modified])

  return links


In [ ]:
cols = ['url']

for index, row in df.iterrows():

  title = row['title']['de']
  #print(title)
  dataset_id = row['id']

  for col in cols:

    val = row[col]
    urls = extractor.find_urls(val)

    for i in urls:
      print(i)
      if 'https' not in i:
        check_url(i)

In [ ]:
url_modified    = ''
# only store links which are faulty or need a protocol update (https)
needsToBeListed = False

if 'https' in url:
  working = check_url(url)

  if not working:
    needsToBeListed = True


else:
  
  url_modified = addHTTPS(url)
  working = check_url(url_modified)

  if working:
    needsToBeListed = True
    
  else:
    # does not change http: urls
    url_modified = addHTTPS(url)
    working = check_url(url)

    if not working:
      url_modified = ''
      needsToBeListed = True

if not needsToBeListed:
  donotign = 0

#
  

In [ ]:

links = []

for url in urls:

  url_modified    = ''
  # only store links which are faulty or need a protocol update (https)
  needsToBeListed = False

  if 'https' in url:
    working = check_url(url)

    if not working:
      needsToBeListed = True


  else:
    
    url_modified = addHTTPS(url)
    working = check_url(url_modified)

    if working:
      needsToBeListed = True
      
    else:
      # does not change http: urls
      url_modified = addHTTPS(url)
      working = check_url(url)

      if not working:
        url_modified = ''
        needsToBeListed = True

  if not needsToBeListed:
    continue
  

  links.append([url, url_modified])


In [71]:
def fromDictionary(d):

    changed  = False

    for key in d.keys():

        val      = d[key]
        isString = True
        

        if type(val) not in [dict, str, list]:
            continue
        elif type(val) != str:
            isString = False
            val = str(val)

        urls = extractor.find_urls(val)

        if len(urls) == 0:
            continue
        else:
            for url in urls:

                working = False

                if not any(prefix in url for prefix in ['www', 'http']):
                    continue
                elif 'https://' in url:
                    continue
                else:
                    #first check if the adjusted url works, then replace it in the original

                    working, adjusted_url = testAdjustedURL(url)

                    if working:
                        changed = True

                        if isString:
                            d[key] = val.replace(url, adjusted_url)
                        else:
                            d[key] = ast.literal_eval(val.replace(url, adjusted_url))


    if changed:
        return [d]
    else:
        return []


In [72]:
def fromList(l):

    changed  = False

    for i in range(len(l)):

        val = l[i]
        isString = True

        if type(val) not in [dict, str, list]:
            continue
        elif type(val) != str:
            isString = False
            val = str(val)

        urls = extractor.find_urls(val)

        if len(urls) == 0:
            continue
        else:
            for url in urls:

                working = False

                if not any(prefix in url for prefix in ['www', 'http']):
                    continue
                elif 'https://' in url:
                    continue
                else:
                    #first check if the adjusted url works, then replace it in the original

                    working, adjusted_url = testAdjustedURL(url)

                    if working:
                        changed = True

                        if isString:
                            l[i] = val.replace(url, adjusted_url)
                        else:
                            l[i] = ast.literal_eval(val.replace(url, adjusted_url))


    if changed:
        return [l]
    else:
        return []


In [73]:
def fromString(string):
     
     changed = False
     urls = extractor.find_urls(string)
     
     for url in urls:
          
          working = False
          
          if not any(prefix in url for prefix in ['www', 'http']):
               continue
          elif 'https://' in url:
               continue

          # change to https://
          else:
               working, adjusted_url = testAdjustedURL(url)
            
          
          if working:
               changed = True
               string  = string.replace(url, adjusted_url)
               
     
     if changed:
          return [string]
     else:
          return []

In [74]:
def getInfo(elem):

    if type(elem) == dict:
        return fromDictionary(elem)

    elif type(elem) == list:
        return fromList(elem)

    elif type(elem) == str:
        return fromString(elem)

    else:
        return []

In [75]:
cols = df.columns

packageIDs      = []
packageNames    = []
attributeNames  = []
attributeValues = []

for index, row in df.iterrows():
    for col in cols:

        value = getInfo(row[col])

        if value == []:
            continue
        
        else:
            packageIDs.append(row['id'])
            packageNames.append(row['title']['de'])
            attributeNames.append(col)
            attributeValues = attributeValues + value

In [84]:
frame = pd.DataFrame([packageIDs, packageNames, attributeNames, attributeValues]).T
frame.columns = ['package_id', 'package_name', 'attribute_name', 'attribute_value']
frame.to_csv('linkAndAttributeValues.csv', index=False)